In [1]:
import PySimpleGUI as sg
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import csv
sg.theme('LightBlue2')

'LightBlue2'

In [2]:
class 項目:
    #インプットファイル読み込み
    def ファイル読み込み(self, シート名):
        layout = [
            [sg.FileBrowse('ファイル選択[xlsx]', font=('Arial',15), key='-FILENAME-'), sg.Text(size=(50,1), font=('Arial',15))],
            [sg.Text(key='-ERROR-', font=('Arial',15))],
            [sg.Button('開く', font=('Arial',15), key='-OPEN-')]]

        window = sg.Window('ファイル選択', layout)

        while True:
            event, values = window.read()
            if event == '-OPEN-':
                if values['-FILENAME-'][-4:] !='xlsx':
                    window['-ERROR-'].update(value='xlsxファイルを選択してください')
                else:
                    print('フォーマットファイル: ',values['-FILENAME-'])
                    break
        window.close()
        df = pd.read_excel(values['-FILENAME-'],sheet_name=f'{シート名}')
        return df
    
    #デフォルト項目
    def 日付追加(self, now, layout):
        layout.append([sg.Text(f'日付: {now.year} 年 {now.month} 月 {now.day} 日 {now.hour} 時 {now.minute} 分', font=('Arial',20), key='日付')])
        return layout
    def 開始ボタン追加(self, layout):
        layout.append([sg.Button('開始', key='開始ボタン', font=('Arial',15)), sg.Text('開始時刻:', key='開始時刻', font=('Arial',20))])
        #layout.append([sg.Text('開始時刻:', key='開始時刻', font=('Arial',20))])
        return layout
    def 終了ボタン追加(self, layout):
        layout.append([sg.Button('終了', key='終了ボタン', font=('Arial',15)), sg.Text('終了時刻:', key='終了時刻', font=('Arial',20))])#, sg.Text(key='所要時間', size=(60,1), font=('Arial',20))])
        #layout.append([sg.Text('終了時刻:', key='終了時刻', font=('Arial',20))])
        layout.append([sg.Text(key='所要時間', size=(60,1), font=('Arial',20))])
        return layout
    def txt保存(self, layout):
        layout.append([sg.Text('保存先:', visible=False, font=('Arial',20), key='savetxt'), sg.InputText(visible=False,key="txt保存先", font=('Arial',20)), sg.FolderBrowse('ブラウザから選択',visible=False, font=('Arial',15), key='txtfolder')])
        layout.append([sg.Text("保存先を選択してください！", visible=False, key='foldererror', size=(60,1), font=('Arial',20))])
        layout.append([sg.Button('保存', key='保存', font=('Arial',15))])
        return layout

    #自由設定項目
    def テキスト入力追加(self, 項目名, layout):
        layout.append([sg.Text(f'{項目名}:', font=('Arial',20)),sg.Push(), sg.InputText(key=f'{項目名}', size=(61,1), font=('Arial',20))])
        return layout
    def 自由記述追加(self, 項目名, height, layout):
        layout.append([sg.Text(f'{項目名}:', font=('Arial',20)),sg.Push(), sg.Multiline(key=f'{項目名}', size=(60,height), font=('Arial',20))])
        return layout
    def ボタン追加(self, 項目名, layout):
        layout.append([sg.Button(f'{項目名}', key=f'{項目名}', font=('Arial',15))])
        return layout
    def プルダウン追加(self, 項目名, リスト, layout):
        layout.append([sg.Text(f'{項目名}:', font=('Arial',20)),sg.Push(), sg.Combo(リスト, key=f'{項目名}', size=(60,1), font=('Arial',20))])
        return layout
    def ブラウザ選択追加(self, 項目名, layout):
        layout.append([sg.Text(f"{項目名}", font=('Arial',20)), sg.FileBrowse('ブラウザから選択', font=('Arial',15), key=f'{項目名}')])
        return layout
    def ファイル名追加(self, 項目名, layout):
        layout.append([sg.Text(f"{項目名}", font=('Arial',20)), sg.InputText(key=f"{項目名}", font=('Arial',20))])
        return layout

In [3]:
class 実験ノート作成(項目):
    def __init__(self, シート名):
        self.comp = super(実験ノート作成, self)
        self.df = self.comp.ファイル読み込み(シート名)
        self.now = now = datetime.datetime.now()
    
    #layout作成
    def 項目追加(self):
        layout = []
        #figlist = []
        項目リスト = []
        for i, title in enumerate(self.df):
            method = self.df[title][0]
            if title == '保存ファイル名':
                txtname = self.df[title][0]
            if title == "日付":
                layout = self.comp.日付追加(self.now, layout)
            if title == "開始ボタン":
                layout = self.comp.開始ボタン追加(layout)
            if title == "終了ボタン":
                layout = self.comp.終了ボタン追加(layout)
            
            if method == "プルダウン":
                pulllist = self.df[title][2:].dropna().values.tolist()
                layout = self.comp.プルダウン追加(title, pulllist, layout)
            elif method == "記述":
                height = self.df[title][1]
                if np.isnan(height):
                    height = 3
                layout = self.comp.自由記述追加(title, height, layout)
            elif method == "テキスト":
                layout = self.comp.テキスト入力追加(title, layout)
            elif method == "ブラウザ":
                layout = self.comp.ブラウザ選択追加(title, layout)
            elif method == "ファイル":
                layout = self.comp.ファイル名追加(title, layout)
        return layout, 項目リスト, txtname

In [4]:
class GUI作成(実験ノート作成):
    def __init__(self, シート名):
        self.note = super(GUI作成, self)
        self.note.__init__(シート名)
        self.layout, self.項目リスト, self.txtname = self.note.項目追加()
    
    def GUI(self):
        #保存ボタンはデフォルトで入れておく
        self.layout.append([sg.Text('保存先:', font=('Arial',20)), sg.FolderBrowse('ブラウザから選択', font=('Arial',15), key='txtfolder')])
        self.layout.append([sg.Button('保存', key='保存', font=('Arial',15))])
        
        columns = sg.Column(self.layout, size=(900, 750), scrollable=True) #columnにしておくとスクロールできる
        layout = [[columns]]
        window = sg.Window('実験ノート', layout, size=(900,750), resizable=True, finalize=True, text_justification='left')
    
        while True:
            event, values = window.read()
            if event == '開始ボタン':
                start = datetime.datetime.now()
                print(f'開始時刻: {start}')
                window['開始時刻'].update(value=f'開始時刻:　{start}')
            if event == '終了ボタン':
                stop = datetime.datetime.now()
                print(f'終了時刻: {stop}')
                print(f'所要時間:　{stop - start}')
                window['終了時刻'].update(value=f'終了時刻:　{stop}')
                window['所要時間'].update(value=f'所要時間:　{stop - start}')
                        
            if event == '保存':
                now = self.now
                txtname = self.txtname
                filename = f'{txtname}_{now.year}{now.month}{now.day}_{now.hour}{now.minute}.txt'
                with open(f'{values["txtfolder"]}/{filename}', 'w') as f:
                    for i, title in enumerate(self.df):
                        method = self.df[title][0]
                        if title == "保存ファイル名":
                            print(f'{values["txtfolder"]}/{filename}\n')
                        elif title == "日付":
                            f.write(f'日付: {now.year} 年 {now.month} 月 {now.day} 日 {now.hour} 時 {now.minute} 分\n')
                            print(f'日付: {now.year} 年 {now.month} 月 {now.day} 日 {now.hour} 時 {now.minute} 分\n')
                        elif title == "開始ボタン":
                            f.write(f'開始時刻: {start}\n')
                            print(f'開始時刻: {start}\n')
                        elif title == "終了ボタン":
                            f.write(f'終了時刻: {stop}\n')
                            f.write(f'所要時間:　{stop - start}\n')
                            print(f'終了時刻: {stop}\n')
                            print(f'所要時間:　{stop - start}\n')
                        elif method == "プルダウン":
                            f.write(f'{title}: {values[title]}')
                            print(f'{title}: {values[title]}\n')
                        elif method == "記述":
                            f.write(f'{title}: {values[title]}\n')
                            print(f'{title}: {values[title]}\n')
                        elif method == "テキスト":
                            f.write(f'{title}: {values[title]}\n')
                            print(f'{title}: {values[title]}\n')
                        elif method == "ファイル":
                            f.write(f'{title}: {values[title]}\n')
                            print(f'{title}: {values[title]}\n')
                        elif method == "ブラウザ":
                            f.write(f'{title}: {values[title]}\n')
                            print(f'{title}: {values[title]}\n')
                window.close()
                break
            if event == sg.WIN_CLOSED:
                break
                window.close()

In [ ]:
ノート = GUI作成('フォーマット')
ノート.GUI()